### **Imports Made**:
- Numpy for general purpose tasks (ex: file loads, matrix operations and arrangements, etc)
- sklearn.linear_model for the multiple models tested
- sklearn.model_selection for hyperparameter tuning and SSE calculation

All model selection methods were utilized at some point, but **train_test_split** proved to be the best tool for this situation.

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

### **Data Extraction**:

Input files are loaded into variables *x*, *y* and *x_project*, this last one being from the *x_test* file. Commented below is a subset of code that performs outlier removal of two points who in a previous analysis were thought to be harmful for our model. But later these points did appear to have a degree of importance, especially due to them being similar to a couple of points in the testing samples. What is not commented, is the removal of four features from x_train and x_test, these same features from analysis through lasso revealed to be a little bit harmful to the SSE of our model. Thus, removing said dimensions improves our accuracy while still keeping our model structured and secure.

In [11]:
batch_size=500
epochs=150
lr=0.001
# Load data
x = np.load("Xtrain_Classification1.npy")
y = np.load("ytrain_Classification1.npy")
# 255 is the biggest number ant value of x can take
train_images = (x).astype('float32')/255.0
train_labels = tf.keras.utils.to_categorical(y,2)
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2)
model_MLP = tf.keras.models.Sequential()
model_MLP.add(tf.keras.layers.Dense(16,activation = 'relu',input_dim = 2700))
model_MLP.add(tf.keras.layers.Dense(8,activation = 'relu'))
model_MLP.add(tf.keras.layers.Dense(2,activation = 'softmax'))

model_MLP.summary()

adam = tf.keras.optimizers.Adam(learning_rate = lr)
model_MLP.compile(optimizer = adam,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])
#MLP without early stopping
history = model_MLP.fit(x = X_train,y=y_train,epochs = epochs,batch_size=batch_size,validation_data = (X_val,y_val))
x_project = np.load("Xtest_Classification1.npy")
BATCH_SIZE = x.shape[0]
IMG_SIZE = (28, 28)
images = np.reshape(x,(len(x),28,28,3))
train_dataset = tf.data.Dataset.from_tensor_slices((images, y))
print(train_dataset)

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.axis("off")



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 16)                43216     
                                                                 
 dense_10 (Dense)            (None, 8)                 136       
                                                                 
 dense_11 (Dense)            (None, 2)                 18        
                                                                 
Total params: 43370 (169.41 KB)
Trainable params: 43370 (169.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/150


ValueError: in user code:

    File "/home/user/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/user/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/user/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/user/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/user/.local/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/user/.local/lib/python3.8/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        r